## 事前準備

1. Google Cloud の新しいプロジェクトを作成します。
2. Cloud Shell を開いて、次のコマンドで API を有効化します。
```
gcloud services enable \
  aiplatform.googleapis.com \
  notebooks.googleapis.com \
  cloudresourcemanager.googleapis.com

```
3. 次のコマンドで Vertex AI Workbench のインスタンスを作成します。
```
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
gcloud workbench instances create agent-development \
  --project=$PROJECT_ID \
  --location=us-central1-a \
  --machine-type=e2-standard-2
```

4. クラウドコンソールのナビゲーションメニューから「Vertex AI」→「Workbench」を選択すると、作成したインスタンス agent-develpment があります。インスタンスの起動が完了するのを待って、「JUPYTERLAB を開く」をクリックしたら、「Python 3(ipykernel)」の新規ノートブックを作成します。

5. この後は、ノートブック上で以下のコマンドを実行していきます。

## ADK パッケージのインストール

In [ ]:
%pip install --upgrade --user \
    google-adk==1.4.1 \
    google-genai==1.20.0 \
    google-cloud-aiplatform==1.97.0

インストールしたパッケージを利用可能にするために、次のコマンドでカーネルを再起動します。

再起動を確認するポップアップが表示されるので [Ok] をクリックします。

In [2]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Search Agent App の作成

In [1]:
import copy, datetime, json, os, pprint, time, uuid
import vertexai
from google.genai.types import Part, Content
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import google_search

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(
    project=PROJECT_ID, location=LOCATION,
    staging_bucket=f'gs://{PROJECT_ID}'
)
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [2]:
instruction = '''
You are a friendly AI assistant that answers user's queries.
Use google_search to give answers based on the latest and objective information.

[Format instruction]
Output in Japanese, in plain text only.
Avoid adding citation marks such as [1][2].
'''

search_agent = LlmAgent(
    name='search_agent',
    model='gemini-2.0-flash-001',
    description='Agent to answer questions using Google Search.',
    instruction=instruction,
    tools=[google_search]
)

In [3]:
class LocalApp:
    def __init__(self, agent, app_name, user_id):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## 実行例

In [5]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

高田馬場でおすすめのカレー屋さんについて調べてみます。


高田馬場にはたくさんのカレー屋さんがあって迷いますよね。いくつかおすすめのお店を紹介します。

*   **横浜ボンベイ:** サラッとしたルーが特徴で、特に「カシミールカレー」が長年愛されています。辛口ですが、野菜の甘みとスパイスの爽快感が絶妙です。
*   **エチオピア:** こちらも有名店で、トロトロに煮込まれた野菜が美味しいカレーが楽しめます。
*   **ブラザー:** 鯖キーマカレーが人気のお店です。
*   **プネウマカレー:** コスパが良いと評判です。
*   **白カレーの店 1/f ゆらぎ:** 珍しい白カレーが味わえます。

食べログなどのランキングも参考にしてみてください。



SessionService が管理するセッション情報を確認します。

セッションを特定するのに必要な、`app_name`、`user_id`、`session_id` を確認します。

In [6]:
app_name = client._session.app_name
user_id = client._session.user_id
session_id = client._session.id

app_name, user_id, session_id

('Search Agent App', 'user00', '7cece7e17955432e9b66383ce66a4b5b')

確認した情報を使って、セッションに保存された情報を確認します。

2 つのイベント（ユーザーの質問とエージェントの回答）が記録されています。

エージェントの回答を生成する際に用いた Google Serch の情報なども確認できます。

In [7]:
session = await client._runner.session_service.get_session(
    app_name = app_name,
    user_id = user_id,
    session_id = session_id,
)

session

Session(id='7cece7e17955432e9b66383ce66a4b5b', app_name='Search Agent App', user_id='user00', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-742a61c4-b920-4f61-95e3-9923e58bd151', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='gwF9QeLt', timestamp=1749248008.635024), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, execu

次のように、これまでのやり取りが「イベント」として記録されています。

In [8]:
def format_timestamp(ts):
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S.%f')

for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-06-06 22:13:28.635024 user
2025-06-06 22:13:28.635763 search_agent


特に、下記の要素から Google 検索に使ったキーワードが確認できます。

In [9]:
from IPython.display import display, HTML

search_queries = session.events[-1].grounding_metadata.web_search_queries
entry_point_html = session.events[-1].grounding_metadata.search_entry_point.rendered_content

print(search_queries)
display(HTML(entry_point_html))

['高田馬場 カレー 人気', '高田馬場 カレー ランキング', '高田馬場 おすすめ カレー']


続けて質問すると、セッション情報を踏まえた回答が得られます。

In [10]:
query = '''
特に家族連れにおすすめなのは？
'''
_ = await client.stream(query)

家族連れにおすすめの高田馬場のカレー屋さんについて調べてみます。


家族連れにおすすめとなると、お店の雰囲気やメニューのバリエーション、子供向けメニューの有無などが重要になりますね。

食べログなどの情報から、子連れで行きやすいとされているお店をいくつかピックアップします。

*   **大戸屋ごはん処 高田馬場駅前店:** 定食屋さんですが、「お子様野菜カレー」があります。アレルギー情報も開示されているので安心です。
*   **プネウマカレー:** コスパが良いという評判で、子供連れでも入りやすいかもしれません。
*   **フードウェイ 高田馬場店:** こちらは本格的なインドカレーのお店で、家族連れの来店も多いようです。チーズナンなど、子供が喜びそうなメニューもあります。

これらの情報を参考に、お店のウェブサイトや口コミなども確認して、お子さんと一緒に楽しめるカレー屋さんを選んでみてください。



先ほどと同様にセッションに記録されたイベントを確認すると、新しい質問と回答のイベントが追加されています。

In [11]:
session = await client._runner.session_service.get_session(
    app_name = app_name,
    user_id = user_id,
    session_id = session_id,
)

for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-06-06 22:13:28.635024 user
2025-06-06 22:13:28.635763 search_agent
2025-06-06 22:13:47.995505 user
2025-06-06 22:13:47.996168 search_agent


In [12]:
search_queries = session.events[-1].grounding_metadata.web_search_queries
entry_point_html = session.events[-1].grounding_metadata.search_entry_point.rendered_content

print(search_queries)
display(HTML(entry_point_html))

['高田馬場 カレー 家族連れ', '高田馬場 カレー 子連れ', '高田馬場 カレー 子供 おすすめ']


## システムインストラクションの確認

ちょっとした裏技を使って、LlmAgent オブジェクトが Gemini API に送っている生の情報を覗いてみます。

In [13]:
import pprint
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator
from google.adk.events.event import Event
from google.adk.models.llm_request import LlmRequest

async def _run_one_step_async(
      self,
      invocation_context: InvocationContext,
  ) -> AsyncGenerator[Event, None]:
    """One step means one LLM call."""
    llm_request = LlmRequest()

    # Preprocess before calling the LLM.
    async for event in self._preprocess_async(invocation_context, llm_request):
        yield event
    if invocation_context.end_invocation:
        return

    # Calls the LLM.
    model_response_event = Event(
        id=Event.new_id(),
        invocation_id=invocation_context.invocation_id,
        author=invocation_context.agent.name,
        branch=invocation_context.branch,
    )

    ## DEBUG output
    if DEBUG:
        print('## Prompt contents ##')
        pprint.pp(llm_request.contents)
        print('----')
        print('## System instruction ##')
        print(llm_request.config.system_instruction)
        print('----')
        print('## Tools config ##')
        pprint.pp(llm_request.config.tools)
        print('----')
    ####

    async for llm_response in self._call_llm_async(
        invocation_context, llm_request, model_response_event
    ):
        # Postprocess after calling the LLM.
        async for event in self._postprocess_async(
            invocation_context, llm_request, llm_response, model_response_event
        ):
            yield event

from google.adk.flows.llm_flows.base_llm_flow import BaseLlmFlow
_run_one_step_async_original = copy.deepcopy(BaseLlmFlow._run_one_step_async)
BaseLlmFlow._run_one_step_async = _run_one_step_async

In [14]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

DEBUG = True
query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

## Prompt contents ##
[Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user')]
----
## System instruction ##

You are a friendly AI assistant that answers user's queries.
Use google_search to give answers based on the latest and objective information.

[Format instruction]
Output in Japanese, in plain text only.
Avoid adding citation marks such as [1][2].


You are an agent. Your internal name is "search_agent".

 The description about you is "Agent to answer questions using Google Search."
----
## Tools config ##
[Tool(function_declarations=None, retrieval=None, google_search=GoogleSearch(time_range_filter=None), google_search_retrieval=None, enterprise_web_search=None, google_maps=None, url_context=None, code_execution=None)]
----
高田馬場でおすすめのカレー屋さんについて調べてみます。


高田馬場にはたくさんのカレー屋さんがあり、それぞれに特徴があり

In [15]:
query = '''
特に家族連れにおすすめなのは？
'''
_ = await client.stream(query)

## Prompt contents ##
[Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='高田馬場でおすすめのカレー屋さんについて調べてみます。\n\n'), Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='高田馬場にはたくさんのカレー屋さんがあり、それぞれに特徴があります。おすすめのお店をいくつかご紹介します。\n\n*   **カレーライス専門店ブラザー:** 鯖キーマカレーが圧倒的な人気を誇ります。\n*   **横浜ボンベイ 高田馬場店:** サラッとしたルーが特徴で、特にカシミールカレーは長年愛される超有名カレーです。\n*   **カリーライス専門店エチオピア 高田馬場店:** 長時間煮込んだたっぷり野菜が美味しいカレーを提供しています。\n*   **プネウマカレー:** コスパ最強の人気店として知られています。\n*    **まっちゃん

`## Prompt contents ##` の部分を見ると、セッションに記録されたイベントをすべてプロンプトに含めるのではなく、コンテキスト（これまでの会話の流れ）を把握するのに必要な情報が選択されていることがわかります。

裏技で変更した内部関数を元に戻しておきます。

In [16]:
DEBUG = False
BaseLlmFlow._run_one_step_async = _run_one_step_async_original

## AgentEngine へのデプロイ

In [17]:
from vertexai import agent_engines

remote_agent = agent_engines.create(
    agent_engine=search_agent,
    display_name='search_agent',
    requirements=[
        'google-adk==1.4.1',
        'google-cloud-aiplatform==1.97.0',
        'google-genai==1.20.0'
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'pydantic': '2.11.5', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.97.0'}
The following requirements are missing: {'pydantic', 'cloudpickle'}
The following requirements are appended: {'cloudpickle==3.1.1', 'pydantic==2.11.5'}
The final list of requirements: ['google-adk==1.4.1', 'google-cloud-aiplatform==1.97.0', 'google-genai==1.20.0', 'cloudpickle==3.1.1', 'pydantic==2.11.5']
Using bucket etsuji-15pro-poc
Wrote to gs://etsuji-15pro-poc/agent_engine/agent_engine.pkl
Writing to gs://etsuji-15pro-poc/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://etsuji-15pro-poc/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/879055303739/locations/us-central1/reasoningEngines/1643154157013565440/operations/7370126874497777664
View progress and logs at https://console.cloud.google.com/logs/query?project=etsuji-15

In [18]:
class RemoteApp:
    def __init__(self, remote_agent, user_id):
        self._remote_agent = remote_agent
        self._user_id = user_id
        self._session = remote_agent.create_session(user_id=self._user_id)
    
    def _stream(self, query):
        events = self._remote_agent.stream_query(
            user_id=self._user_id,
            session_id=self._session['id'],
            message=query,
        )
        result = []
        for event in events:
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

    def stream(self, query):
        # Retry 4 times in case of resource exhaustion 
        for c in range(4):
            if c > 0:
                time.sleep(2**(c-1))
            result = self._stream(query)
            if result:
                return result
            if DEBUG:
                print('----\nRetrying...\n----')
        return None # Permanent error

In [19]:
remote_client = RemoteApp(remote_agent, 'user00')

query = '''
今年のゴールデンウィークは、何連休でしょうか？
'''
_ = remote_client.stream(query)

 2025年のゴールデンウィークは、暦通りだと5月3日（土）から5月6日（火）までの4連休です。

しかし、4月28日（月）と4月30日（水）から5月2日（金）の平日3日間を休みにすると、最大11連休になります。



In [20]:
query = '''
来年はどうなりますか？
'''
_ = remote_client.stream(query)

 2026年のゴールデンウィークは、5月2日（土）から5月6日（水）までの5連休です。4月30日（木）と5月1日（金）の2日間を休むと、4月29日（水）から5月6日（水）まで最大8連休になります。



セッション情報を参照する際は、`VertexAiSessionService` を使用します。

In [21]:
app_name = remote_client._session['appName']
user_id = remote_client._session['userId']
session_id = remote_client._session['id']

app_name, user_id, session_id

('1643154157013565440', 'user00', '3578844927469027328')

In [22]:
from google.adk.sessions import VertexAiSessionService

session_service = VertexAiSessionService(
    project = PROJECT_ID,
    location = LOCATION,
)

session = await session_service.get_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id,
)

session

Session(id='3578844927469027328', app_name='1643154157013565440', user_id='user00', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='\n今年のゴールデンウィークは、何連休でしょうか？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-5e09b04e-08dc-4149-b057-faeabdb0bf47', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='7383100183992598528', timestamp=1749248256.937505), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=Non

In [23]:
for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-06-06 22:17:36.937505 user
2025-06-06 22:17:37.144144 search_agent
2025-06-06 22:17:39.468261 user
2025-06-06 22:17:39.663275 search_agent


In [25]:
search_queries = session.events[1].grounding_metadata.web_search_queries
entry_point_html = session.events[1].grounding_metadata.search_entry_point.rendered_content

print(search_queries)
display(HTML(entry_point_html))

['2025年 ゴールデンウィーク 何連休', '2025年 ゴールデンウィーク 期間']


In [26]:
search_queries = session.events[3].grounding_metadata.web_search_queries
entry_point_html = session.events[3].grounding_metadata.search_entry_point.rendered_content

print(search_queries)
display(HTML(entry_point_html))

['2026 年の祝日', '2026 ゴールデンウィーク 休み', '2026 ゴールデンウィーク']


## 後片付け

デプロイしたエージェントを削除します。

In [27]:
remote_agent.delete(force=True)

Delete Agent Engine backing LRO: projects/879055303739/locations/us-central1/operations/3668167980799229952
Agent Engine deleted. Resource name: projects/879055303739/locations/us-central1/reasoningEngines/1643154157013565440


デプロイ済みのエージェントをまとめて削除する場合は、次を実行します。

In [ ]:
for agent in agent_engines.list():
    print(f'{agent.display_name}, {agent.gca_resource.name}')
    agent.delete(force=True)